### Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import requirements

In [ ]:
from collections import Counter
import torch
from torch import nn, optim
from torch.utils import data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Load data

In [ ]:
datapath = r'/content/drive/MyDrive/dogecoin/doge_usdt_4_1_5_18.csv'
dataset = pd.read_csv(datapath, header=None)

In [ ]:
dataset.head()

### Features and labels


In [ ]:
id_c = 0
high_c = 1
low_c = 2
open_c = 3
close_c = 4
count_c = 5
amount_c = 6
volume_c = 7

increase = 1
decrease = 0

time_series = 60

In [ ]:
feature_dataset = dataset.drop(id_c, axis=1)

In [ ]:
feature_dataset.head()

In [ ]:
features = []
labels = []

for i in range(time_series, len(feature_dataset)):
  full_temp = feature_dataset.loc[i - time_series:i]
  feature_temp = full_temp.iloc[0:time_series]
  feature_temp = feature_temp.values
  features.append(feature_temp)

  open_temp = full_temp.iloc[-1, open_c - 1]
  close_temp = full_temp.iloc[-1, close_c - 1]
  label_temp = increase if open_temp <= close_temp else decrease
  labels.append(label_temp)

features = np.array(features)
labels = np.array(labels)

In [ ]:
print(features.shape)
print(labels.shape)

### Train dataset and test dataset

In [ ]:
test_full_ratio = 0.2
random_seed = 15

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=test_full_ratio, random_state=random_seed)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

### Data to torch

In [ ]:
train_x = torch.from_numpy(train_x)
train_y = torch.from_numpy(train_y)

test_x = torch.from_numpy(test_x)
test_y = torch.from_numpy(test_y)

### Batch size

In [ ]:
BATCH_SIZE = 128

torch_train_dataset = data.TensorDataset(train_x, train_y)
train_iterator = data.DataLoader(
  dataset = torch_train_dataset,
  batch_size = BATCH_SIZE
)

torch_test_dataset = data.TensorDataset(test_x, test_y)
test_iterator = data.DataLoader(
  dataset = torch_test_dataset,
  batch_size = BATCH_SIZE
)

### LSTM

In [ ]:
class RNN_Model(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super(RNN_Model, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim
    self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
    self.fc = nn.Linear(hidden_dim, output_dim)
      
  def forward(self, x):
    # (layer_dim, batch_size, hidden_dim)
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)
    out, hn = self.rnn(x, h0.detach())
    out = self.fc(out[:, -1, :])
    return out

### Init

In [ ]:
input_dim = 7
hidden_dim = 256
layer_dim = 2
output_dim = 2

model = RNN_Model(input_dim, hidden_dim, layer_dim, output_dim)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Loss

In [ ]:
criterion = nn.CrossEntropyLoss()

### Optimizer

In [ ]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Train

In [ ]:
sequence_dim = time_series
loss_list = []
accuracy_list = []
iteration_list = []

EPOCHS = 10
iter = 0
print_times = 50
for epoch in range(EPOCHS):
  for i, (fea, lab) in enumerate(train_iterator):
    model.train()
    fea = fea.view(-1, sequence_dim, input_dim).requires_grad_().float().to(device) 
    lab = lab.to(device)

    optimizer.zero_grad()
    outputs = model(fea)
    loss = criterion(outputs, lab)
    loss.backward()

    optimizer.step()
    iter += 1

    if iter % print_times == 0:
      model.eval()
      correct = 0.0
      total = 0.0

      for fea, lab in test_iterator:
        fea = fea.view(-1, sequence_dim, input_dim).float().to(device) 
        outputs = model(fea)
        predict = torch.max(outputs.data, 1)[1]
        total += lab.size(0)

        if torch.cuda.is_available():
          correct += (predict.gpu() == lab.gpu()).sum()
        else:
          correct += (predict == lab).sum()

      accuracy = correct / total * 100
      loss_list.append(loss.data)
      accuracy_list.append(accuracy)
      iteration_list.append(iter)

      print("loop : {}, Loss : {}, Accuracy : {}".format(iter, loss.item(), accuracy))

### Loss

In [ ]:
plt.plot(iteration_list, loss_list)
plt.xlabel('Number of Iteration')
plt.ylabel('Loss')
plt.title('RNN')
plt.show()

### Accuracy

In [ ]:
plt.plot(iteration_list, accuracy_list, color='r')
plt.xlabel('Number of Iteration')
plt.ylabel('Accuracy')
plt.title('LSTM')
plt.savefig('LSTM_mnist.png')
plt.show()